In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
import src.config as config

In [3]:
from datetime import datetime, timedelta, timezone
import pandas as pd

current_date = pd.to_datetime(datetime.now(timezone.utc)).floor('h')
print(f'{current_date=}')
# current_date=Timestamp('2026-01-08 23:00:00+0000', tz='UTC')

current_date=Timestamp('2026-01-09 05:00:00+0000', tz='UTC')


In [4]:
from src.inference_1 import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

c:\Users\HOME\anaconda3\envs\pyimg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2026-01-08 23:23:17,289 INFO: Initializing external client
2026-01-08 23:23:17,292 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-08 23:23:19,432 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1329302
Fetching all available data from feature store
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (17.23s) 


In [5]:
from src.inference_1 import(
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2026-01-08 23:23:42,219 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-08 23:23:42,232 INFO: Initializing external client
2026-01-08 23:23:42,233 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-08 23:23:42,953 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1329302


Downloading: 100.000%|██████████| 789586/789586 elapsed<00:00 remaining<00:00


[LightGBM] [Warning] feature_fraction is set=0.3201254152687832, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3201254152687832
[LightGBM] [Warning] bagging_fraction is set=0.60384981778599, subsample=1.0 will be ignored. Current value: bagging_fraction=0.60384981778599


In [6]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2026-01-09 05:00:00+00:00
1,2,0.0,2026-01-09 05:00:00+00:00
2,3,1.0,2026-01-09 05:00:00+00:00
3,4,2.0,2026-01-09 05:00:00+00:00
4,5,0.0,2026-01-09 05:00:00+00:00
...,...,...,...
255,261,2.0,2026-01-09 05:00:00+00:00
256,262,12.0,2026-01-09 05:00:00+00:00
257,263,13.0,2026-01-09 05:00:00+00:00
258,264,2.0,2026-01-09 05:00:00+00:00


Save these predictions in the feature store, so that they can be later consumed by our Streamlit app

In [7]:
from src.feature_store_api import get_feature_store

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name= config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated by our production model",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
)

2026-01-08 23:23:53,117 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-08 23:23:53,127 INFO: Initializing external client
2026-01-08 23:23:53,129 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-08 23:23:53,924 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1329302


In [8]:
feature_group.insert(predictions, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 260/260 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1329302/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions
2026-01-08 23:24:08,502 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-08 23:24:14,799 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-08 23:25:51,768 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-08 23:25:51,864 INFO: Waiting for log aggregation to finish.
2026-01-08 23:26:06,858 INFO: Execution finished successfully.


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)